#  Matching Users

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from Utils import TransantiagoConstants

first_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/1st_quarter.xlsx"
second_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/2nd_quarter.xlsx"
third_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/3rd_quarter.xlsx"
codes_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/04_DTPM/codes_services.xlsx"

first_quarter_evasion = pd.read_excel(first_quarter_path, encoding = 'latin-1')
second_quarter_evasion = pd.read_excel(second_quarter_path, encoding = 'latin-1')
third_quarter_evasion = pd.read_excel(third_quarter_path, encoding = 'latin-1')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

first_quarter_evasion['TIEMPO'] = first_quarter_evasion['HORA'].astype(str)+':'+first_quarter_evasion['MINUTOS'].astype(str)+':00'
second_quarter_evasion['TIEMPO'] = second_quarter_evasion['HORA'].astype(str)+':'+second_quarter_evasion['MINUTOS'].astype(str)+':00'
third_quarter_evasion['TIEMPO'] = third_quarter_evasion['HORA'].astype(str)+':'+third_quarter_evasion['MINUTOS'].astype(str)+':00'

first_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
second_quarter_evasion.columns = ['FECHA', 'SERVICIO','TIPO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
third_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
del second_quarter_evasion['TIPO']

#Processing codes before merging.

codes_ida = codes[codes['DIRECTION']=='Ida']
codes_ret = codes[codes['DIRECTION']=='Ret']

codes_ida = codes_ida.rename(columns = {'USER_CODE':'SERVICIO'})
codes_ret = codes_ret.rename(columns = {'USER_CODE':'SERVICIO'})

del codes_ida['DIRECTION']
del codes_ret['DIRECTION']

In [ ]:
print(type(first_quarter_evasion.loc[0,'TIEMPO'])) #should be a string
print(type(first_quarter_evasion.loc[0,'FECHA'])) #should be a timestamp

In [ ]:
frames = [first_quarter_evasion, second_quarter_evasion, third_quarter_evasion]
evasion = pd.concat(frames, keys=['first', 'second', 'third'])
del evasion['LUGAR_INICIO']

* Merge should be made only by codes_ida or codes_ret. Be aware of it

In [ ]:
evasion = pd.merge(evasion,codes_ida, on=['SERVICIO'], how='left')

In [ ]:
print(evasion.loc[evasion['TS_CODE'].isnull(),'SERVICIO'].unique()) #should be only D06, so the line below makes sense. Anyway, it is hardcoded so be aware.
evasion.loc[evasion['TS_CODE'].isnull(),'TS_CODE']='446'

* Processing evasion-ddbb before merging

In [ ]:
evasion['PATENTE'] =  evasion['PATENTE'].str.replace(' ','')

evasion['SERVICIO_TMP'] = evasion['SERVICIO'].apply(str)
evasion['TS_CODE_TMP'] = evasion['TS_CODE'].apply(str)

del evasion['SERVICIO']
del evasion['TS_CODE']

evasion = evasion.rename(columns = {'SERVICIO_TMP':'SERVICIO_USUARIO', 'TS_CODE_TMP':'SERVICIO'})
evasion_paradero = evasion[evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]
print(type(evasion['SERVICIO'][0])) #should be a string

In [ ]:
print('Number of rows in complete evasion database is: ' + str(len(evasion.index)))
print('Number of rows in evasion in paradero database is: ' + str(len(evasion_paradero.index)))
print('Number of rows in evasion in paradero by first door database is: ' + str(len(evasion_paradero_first.index)))

* Getting rid of duplicated rows

In [ ]:
#Getting duplicated rows
duplicated_evasion_paradero_first = evasion_paradero_first[evasion_paradero_first.duplicated(['FECHA','PATENTE','PUERTAS','N_PUERTA','HORA_INICIO','HORA','MINUTO','TP','TIEMPO','UN','SERVICIO_USUARIO','SERVICIO'], keep=False)]
print('Number of duplicated rows in evasion in paradero by first door database is: ' + str(len(duplicated_evasion_paradero_first.index)))

#Colapsing duplicated rows
grouped_temporal_evasion_paradero_first = duplicated_evasion_paradero_first.groupby(['FECHA','PATENTE','PUERTAS','N_PUERTA','HORA_INICIO','HORA','MINUTO','TP','TIEMPO','UN','SERVICIO_USUARIO','SERVICIO'])['INGRESAN','NO_VALIDAN'].sum()
grouped_temporal_evasion_paradero_first = grouped_temporal_evasion_paradero_first.reset_index()
print('Number of collapsed-duplicated rows in evasion in paradero by first door database is: ' + str(len(grouped_temporal_evasion_paradero_first.index)))

#Deleting duplicated rows in evasion in paradero by first door database
non_duplicated_evasion_paradero_first = evasion_paradero_first.drop_duplicates(['FECHA','PATENTE','PUERTAS','N_PUERTA','HORA_INICIO','HORA','MINUTO','TP','TIEMPO','UN','SERVICIO_USUARIO','SERVICIO'], keep=False)
print('Number of rows in evasion in paradero by first door without duplicated rows at all is: ' + str(len(non_duplicated_evasion_paradero_first.index)))

#Appendind
del evasion_paradero_first
frames = [non_duplicated_evasion_paradero_first, grouped_temporal_evasion_paradero_first]
evasion_paradero_first = pd.concat(frames)
print('Final number of rows in evasion in paradero by first door with collapsed duplicated rows is: ' + str(len(evasion_paradero_first.index)))

* Filtering dates not in common dates

In [ ]:
common_dates = TransantiagoConstants.common_dates
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]
common_dates_evasion = evasion_paradero_first[evasion_paradero_first['FECHA'].isin(common_dates_timestamp)]
len(common_dates_evasion.index) #Remember to store the number.

* <font color='red'>Filtering for one specific common date and analyze it to refactor and debug <- this part should be omitted in the final script. Above methodology is general</font>

In [ ]:
example = common_dates_evasion[common_dates_evasion['FECHA']==pd.to_datetime('2017-04-11')]

In [ ]:
len(example.index)

In [ ]:
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [ ]:
date = '2017-04-11'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
sorted_df = etapas_builder.runRawProcess()

In [ ]:
len(sorted_df.index)

* Processing sorted_df before reducing complexity

In [ ]:
sorted_df['servicio_subida'] =  sorted_df['servicio_subida'].str.replace('T','')
sorted_df['servicio_subida'] =  sorted_df['servicio_subida'].str.split(' ').str[0]

In [ ]:
sorted_df['sitio_subida'] = sorted_df['sitio_subida'].str.replace("-", "")
sorted_df['sitio_subida'] = sorted_df['sitio_subida'].str.replace(" ", "")

* First: filter etapas by patentes and servicios surveyed in evasion ddbb so to reduce search space

In [ ]:
patentes = example['PATENTE'].unique()
patentes = patentes.tolist()
servicios = example['SERVICIO'].unique()
servicios = servicios.tolist()

In [ ]:
clean_sorted_df = sorted_df[(sorted_df['sitio_subida'].isin(patentes))&(sorted_df['servicio_subida'].isin(servicios))]

In [ ]:
sorted_df.head()

In [ ]:
len(clean_sorted_df.index)

* Evasion should be sorted first

In [ ]:
example['TIEMPO'] = example['FECHA'].dt.strftime('%Y-%m-%d') + ' ' + example['TIEMPO']

In [ ]:
example['TIEMPO'] = pd.to_datetime(example['TIEMPO'])

In [ ]:
example = example.sort_values(by=['PATENTE', 'SERVICIO', 'TIEMPO'], ascending=[True, True, True])

In [ ]:
example = example.reset_index(drop =True)

In [ ]:
example.head(8)

In [ ]:
clean_sorted_df[(clean_sorted_df['sitio_subida']=='BJFB62')&(clean_sorted_df['servicio_subida']=='507')&(clean_sorted_df['t_subida']>=pd.to_datetime('2017-04-11 15:08:00'))].head(15)

In [ ]:
clean_sorted_df = clean_sorted_df.reset_index(drop=True)

In [ ]:
clean_sorted_df.head()

* New Methodology: this methodology is based on appending PARADEROS to evasion ddbb

In [ ]:
example.head()

In [ ]:
i = 0
past_plate = '-'
past_service = '-'

example['start_cut']=''
example['end_cut']=''
example['paradero']=''
past_util_df = pd.DataFrame()
past_par_subida = ['']
from IPython.core.debugger import Tracer

for index,row in example.iterrows():
    future_index = index+1
    actual_plate = row['PATENTE']
    actual_service = row['SERVICIO']
    actual_hour = row['TIEMPO']
    
    if (actual_plate!=past_plate)|(actual_service!=past_service):
        past_hour = pd.to_datetime('2017-04-11 00:00:00') #Starting hour <- consider to change this
        
    if past_hour == pd.to_datetime('2017-04-11 00:00:00'): #Assuming a pre-defined value, i.e., 1 minute.
        start_cut = actual_hour - (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        start_cut = actual_hour - (actual_hour - past_hour)/2
    
    if future_index >= len(example.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 1 minute.
        end_cut = actual_hour + (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        future_hour = example.loc[future_index,'TIEMPO']
        end_cut = actual_hour + (future_hour - actual_hour)/2
        
    actual_util_df = clean_sorted_df[(clean_sorted_df['sitio_subida']==actual_plate)&(clean_sorted_df['servicio_subida']==actual_service)&(start_cut<=clean_sorted_df['t_subida'])&(clean_sorted_df['t_subida']<=end_cut)]
    actual_par_subida = actual_util_df['par_subida'].unique().tolist()
    
    if len(actual_par_subida)<1: 
        #Aqui hay dos situaciones: 
        #1. Las trx no alcanzan a estar dentro del rango
        #2. Simplemente hay anotaciones extrañas.
        #First: agrandar el rango de busqueda por 30 segundos...
        start_cut = start_cut - (pd.to_datetime('2017-04-11 00:01:30') - pd.to_datetime('2017-04-11 00:01:00'))
        end_cut = end_cut + (pd.to_datetime('2017-04-11 00:01:30') - pd.to_datetime('2017-04-11 00:01:00'))
        actual_util_df = clean_sorted_df[(clean_sorted_df['sitio_subida']==actual_plate)&(clean_sorted_df['servicio_subida']==actual_service)&(start_cut<=clean_sorted_df['t_subida'])&(clean_sorted_df['t_subida']<=end_cut)]
        actual_par_subida = actual_util_df['par_subida'].unique().tolist()
        #Si agrandar el rango de busqueda no funciona... entonces asignar el anteriormente visitado.
        if len(actual_par_subida)<1:
            actual_par_subida = past_par_subida
            example.loc[index,'paradero']=actual_par_subida[len(actual_par_subida)-1] #Esto es asignar el último visitado (ad-hoc)
        #Si agrandar el rango de busqueda funciona, entonces asignar el paradero que se encontró
        else:
            example.loc[index,'paradero']=actual_par_subida[0]
            past_par_subida = actual_par_subida
    
    elif len(actual_par_subida)==1:
        #Si se encontró un sólo paradero, entonces asignar (solución más sensata)
        example.loc[index,'paradero']=actual_par_subida[0]
        past_par_subida = actual_par_subida
    
    elif (len(actual_par_subida)>1):
        #Si se observa más de un paradero...
        if (past_par_subida[(len(past_par_subida)-1)]==actual_par_subida[0]):
            #Si el último paradero de la iteración anterior coincide con el primer paradero de la iteración actual
            example.loc[index,'paradero']=actual_par_subida[1] #Asignar el siguiente paradero en la lista.
            past_par_subida = actual_par_subida[1]
        else: 
            #Si no coinciden... asignar el primero
            example.loc[index,'paradero']=actual_par_subida[0] #Asignar el primero.
            past_par_subida = actual_par_subida[0]
        
#    if index>=11:
#        Tracer()()
    example.loc[index,'start_cut']=start_cut
    example.loc[index,'end_cut']=end_cut
    past_plate = actual_plate
    past_service = actual_service
    past_hour = actual_hour
    


In [ ]:
example.loc[8:20]

In [ ]:
example.tail(50)

In [ ]:
clean_sorted_df.loc[60:85]

In [ ]:
par_subida

* Old Methodology

In [ ]:
i = 0
past_plate = '-'
past_service = '-'

example['total_delta']=''
example['mean_delta']=''
example['count_delta']=''
example['start_cut']=''
example['end_cut']=''

for index,row in example.iterrows():
    future_index = index+1
    actual_plate = row['PATENTE']
    actual_service = row['SERVICIO']
    actual_hour = row['TIEMPO']
    
    if (actual_plate!=past_plate)|(actual_service!=past_service):
        past_hour = pd.to_datetime('2017-04-11 00:00:00') #Starting hour <- consider to change this
        
    if past_hour == pd.to_datetime('2017-04-11 00:00:00'): #Assuming a pre-defined value, i.e., 1 minute.
        start_cut = actual_hour - (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        start_cut = actual_hour - (actual_hour - past_hour)/2
    
    if future_index >= len(example.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 1 minute.
        end_cut = actual_hour + (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        future_hour = example.loc[future_index,'TIEMPO']
        end_cut = actual_hour + (future_hour - actual_hour)/2  

    util_df = clean_filtered_df[(clean_filtered_df['sitio_subida']==actual_plate)&(clean_filtered_df['servicio_subida']==actual_service)&((start_cut<=clean_filtered_df['t_subida'])&(clean_filtered_df['t_subida']<=end_cut))]
    util_df_indexes = util_df.index.values
    number_of_indexes = len(util_df_indexes)
    
    first_util_df_index = util_df_indexes[0]
    last_util_df_index = util_df_indexes[number_of_indexes-1]
    #Checking up of util_df --- until a condition is reached
    j=1
    while True:
        upper_util_row = clean_filtered_df.loc(first_util_df_index-j)
        if upper_util_row['par_subida'] == util_df['par_subida']: #Si el paradero de subida de la fila inmediatamente anterior al util_df es el mismo, then append it and continue < the issue here es que hay que sacarlo del util_df anteriormente analizado...
            util_df.concat[upper_util_row]
            j=j+1
        else 
    
    
    total_delta = util_df['diferencia_tiempo_secs'].sum()
    mean_delta = util_df['diferencia_tiempo_secs'].mean()
    count_delta = util_df['diferencia_tiempo_secs'].count() 
    
    example.loc[index,'total_delta'] = total_delta
    example.loc[index,'mean_delta'] = mean_delta
    example.loc[index,'count_delta'] = count_delta
    example.loc[index,'start_cut']=start_cut
    example.loc[index,'end_cut']=end_cut
    
    past_plate = actual_plate
    past_service = actual_service
    past_hour = actual_hour
    
    i=i+1
#    if (actual_plate == 'ZN6539')&(actual_service=='109'):
#        print(i)
#        print(start_cut)
#        print(end_cut)

#        break

# TEST OLD METHODOLOGY (Check issue #8)

In [ ]:
example.head(10)

* Above table shows:
- PATENTE == BJFB62 & TIEMPO == 2017-04-11 15:10:00 & SERVICIO == 507 > sum = 08, mean = 2.66667, count = 3 > check them in clean_filtered_df.loc[182369:182380] > <font color='green'>Test passed</font>
- PATENTE == BJFB62 & TIEMPO == 2017-04-11 15:32:00 & SERVICIO == 507 > sum = 13, mean = 6.50000, count = 2 > check them in clean_filtered_df.loc[182388:182395] > <font color='green'>Test passed, although incorrectly assignment</font>

In [ ]:
clean_filtered_df.loc[182369:182380]

In [ ]:
clean_filtered_df.loc[182388:182395]

In [ ]:
example.tail(10)

* Above table shows:
- PATENTE == ZN6539 & TIEMPO == 2017-04-11 14:31:00 & SERVICIO == 109 > sum = 08, mean = 04, count = 02 > check them in clean_filtered_df.loc[2527175:2527180] > <font color='green'>Test passed</font>
- PATENTE == ZN6539	& TIEMPO == 2017-04-11 14:37:00 & SERVICIO == 109 > sum = 21, mean = 21, count = 01 > check them in clean_filtered_df.loc[2527178:2527187] > <font color='green'>Test passed</font>

In [ ]:
clean_filtered_df.loc[2527175:2527180]

In [ ]:
clean_filtered_df.loc[2527178:2527187]

In [ ]:
util_df

In [ ]:
df_indexes = util_df.index.values

In [ ]:
length = len(df_indexes)

In [ ]:
df_indexes[length-1]